# Load Individual Household Electric Power Consumption
https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption

## Readme
- https://medium.com/@ottaviocalzone/an-intuitive-explanation-of-lstm-a035eb6ab42c
- https://www.youtube.com/watch?v=b61DPVFX03I

In [ ]:
import os
import torch
from support import EnergyConsumptionDataset, EnergyConsumptionModule, build_loaders, demo_model_shapes, \
    plot_convergence, train_and_evaluate, plot_predictions_grid, plot_mse_horizon

In [ ]:
data_folder = "local_data/cache"
if not os.path.exists(data_folder):
    os.makedirs(data_folder, exist_ok=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

In [ ]:
all_data = EnergyConsumptionDataset(history_samples=24 * 14, horizon_samples=24 * 3, scale=True, train_fraction=0.9)
print("data length: ", len(all_data))
print("number of channels: ", all_data.number_of_channels)

# Explore dataset

In [ ]:
all_data.data_features.head()

In [ ]:
train_dataset, test_dataset, train_loader, test_loader = build_loaders(all_data, train_fraction=0.9, batch_size_train=200, batch_size_eval=2000)

In [ ]:
model = EnergyConsumptionModule(input_dim=all_data.number_of_channels, output_dim=all_data.horizon_samples)

In [ ]:
demo_model_shapes(model, train_loader)

In [ ]:
checkpoint_file_name = os.path.join(data_folder, "ERAdmission-CNN-LSTM-Fusion.checkpoint.pt")
model, progress_log, test_accuracy, test_loader = train_and_evaluate(model, train_dataset, test_dataset, device=device, n_epochs=1, batch_size=200, lr=1e-3, checkpoint_file_name=checkpoint_file_name)


In [ ]:
plot_convergence(progress_log, test_accuracy)
plot_predictions_grid(model, test_loader, device=device, n=6)
plot_mse_horizon(model, test_loader, all_data.horizon_samples, device=device)

# Prediction Examples
Below are small, focused examples showing how to run predictions with the trained `model` and `test_loader`.
Each code block is a single, logical step: prepare batch, run prediction, compute simple metrics, and plot results.

In [ ]:
# 1) Prepare a small batch for inference
model.eval()
with torch.no_grad():
    X_all, y_all = next(iter(test_loader))
    # move inputs to device if available
    X_all = X_all.to(device) if 'device' in globals() else X_all
    n_samples = min(6, X_all.shape[0])
    X_sample = X_all[:n_samples]
    y_true = y_all[:n_samples].cpu()
    # reduce to primary channel if necessary: (batch, channels, horizon) -> (batch, horizon)
    if y_true.ndim == 3:
        y_true = y_true[:, 0, :]
    print('Prepared', n_samples, 'samples ->', 'X_sample', X_sample.shape, 'y_true', y_true.shape)

In [ ]:
# 2) Run prediction on the small batch and prepare results for plotting
with torch.no_grad():
    y_pred = model(X_sample)
    # model may return shape (batch, channels, horizon) or (batch, horizon)
    if y_pred.ndim == 3:
        y_pred = y_pred[:, 0, :]
    y_pred = y_pred.cpu()
print('y_pred shape', y_pred.shape)

In [ ]:
# 3) Compute simple MSE metrics on the small batch
mse_per_sample = ((y_pred - y_true) ** 2).mean(dim=1)
print('MSE per sample:', mse_per_sample.numpy())
print('Mean MSE (batch):', float(mse_per_sample.mean()))

In [ ]:
# 4) Plot actual vs predicted for the small batch (easy visual check)
import matplotlib.pyplot as plt
rows, cols = 2, 3
plt.figure(figsize=(14, 8))
for i in range(y_pred.shape[0]):
    plt.subplot(rows, cols, i + 1)
    plt.plot(y_true[i], label='Actual', marker='o', markersize=3)
    plt.plot(y_pred[i], label='Predicted', marker='x', markersize=3, alpha=0.8)
    plt.xlabel('Horizon step')
    plt.grid(True, alpha=0.3)
    plt.title(f'Sample {i} — MSE: {mse_per_sample[i]:0.4f}')
    if i == 0:
        plt.legend()
plt.tight_layout()